# Fundamentals of Social Data Science 
## Week 1, Day 3: Merging and aggregating data 

In this walkthrough we are going to cover some elements of reshaping data. Data about the world is for its own purpose, which might not align with the sorts of objects of inquiry or units of analysis that we are most interested in. In particular, data about the world is often hierarchical, with things nested inside other things. For example, people are located in towns and cities which are in countries. We may have measurements about people but want to get measurements about cities or vice versa. 

We will be using some synthetic network data today in the virtual walkthrough because it is a useful case for considering how to link data across different tables and shapes and the Iris data just won't do. Then at the end of the Walkthrough I will be introducing a second feather dataframe from the Movies Stack Exchange. 

With data in multiple tables we can explore three key types of operations: 
- One where we _reorganise_ the data we are working with, 
- One where we _increase_ the amount of data that we are working with, 
- One where we _decrease_ the amount of data that we are working with. 

We will discuss each in turn and then see them in action with our data. 


# Precursor: Aligning data using keys 

Regardless of how we wrangle our data, we will want to know which data pertains to which case. For that we use the notion of a 'key'. Each row in a data has an index by definition, but what we want when we wrangle data is to ensure that there is some way to link data from two different sources. 

We have already seen keys in dictionaries and the Series, as in `key-value` pairs. But the key isn’t simply the first part of the key-value pair, it’s _part_ of the way that we index the data. Typically, we want our data to have a unique identifier for each row. In some approaches this is called the “primary key” and it is supposed to be unique. In fact, in some databases, if you set a column to be a primary key then it will enforce the rule that each row has a unique value. But, let’s consider how we can link data: a person might have a UserID in our system and that person would make posts on a website like SE. So each post would contain its own unique key (in this case the PostID) but it would also contain a value for the user (the UserID). If we add new posts, they should have their own unique identifier (i.e. primary key) but they do not need to have unique UserIDs, because that would mean each user could only make one post. 

These decisions to reshape data in this way occur in some respects because of database ‘normalisation’. This is the feature of a database such that we seek to minimise duplication of data within the store. Thus, every table will have some sort of case in each row (which from our perspective might be a unit of analysis). Then to link the databases we would have keys that link the data together. 

Imagine that we do not have a table for user data but only a table for post data. That would mean we would need to put everything about the user in that table. So we add a new post, then it would need to copy in all the data about that user. They make a second post, then again we would copy all that data about the user. Suddenly we have a lot of redundant data. By having a user table, we have one place so we only need to update the data once. By having a key, we can then link that single canonical table of user data with every post. This is an example of a ‘one-to-many’ relationship. There is one row for the user and many rows of post data that can pertain to that user. 

We can use any column of data as a key so long as we can match that between two tables. However, some forms of data are more amenable to being keys than others. Data such as names end up being poor keys for a few reasons. The obvious one is uniqueness. However, there are also issues with spelling conventions to consider. It is for this reason that we typically do not use country names as keys in data on that scale. Instead, we would use a country code (such as the ISO two or three letter codes for countries). One popular standard that is emerging is the use of UUIDs or Uniform Universal Identifiers which we will see below.  

A UUID is as a 128-bit string done with HEX codes, separated with hyphens for readability. These codes are base-36. How can we do base-36? It's just the 26 letters of the alphabet plus the numbers 0-9. So a UUID would take the following structure: 

xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx

Such as: 

8e63036c-cc56-4b95-b449-196781024be0

It is not necessary to use UUIDs. As you will see in the Stack Exchange data, they simply use digits starting from 0. 


# Reorganising the data

This is where we have the same volume of data but it is now aligned in a different way. This is what happens when we add a row to a DataFrame. We already have the data in two different places and we just combine those cells. We normally want to ensure that our data is _aligned_ in some way. Thus, when we add columns to a DataFrame, the rows pertain to the correct cases or when we add cases, that they have the correct values in the columns. 

Recall that data can be organised by position or by order. When we add data, we similarly can add it in the same order or by matching some keys. When we get results from an analysis, we might end up getting just an array of values that’s in the order the cases went in. Or we might get a dictionary where the key is the index and the value is the new value we want in our DataFrame. 

Which way is more fragile? Of course it is the position-based one. If we resort, filter or otherwise alter our data, the number or order of the cases might not match between the DataFrame and the new list to be imported. 

Generally if we have data in a dictionary we would create it as a Series and then add the Series to our DataFrame. If we have it in a list of the correct size and order we might just say `df[‘var’] = new_list`. If we have two DataFrames we can ‘concatenate’ them which is akin to adding one to the other. Again, we want to ensure that we are aligned with respect to row and column names. Pandas is smart enough to be able to add missing data where there might be a gap if we add things by index. 

To reorganise data: 
- **Operation**: A concatenation (e.g., `pd.concat([<collection_of_tables>])` )
- **Outcome**: A combined table of similar shape from two or more sources
- **Research purpose**: Enable one to analyse more data of the same kind. 
- **Example**: Collecting data on comments from one thread and adding them to data on comments from another thread. They are different threads, but the structure of the data is the same. 
- **Follow-up**: Then we can ask questions across groups with the same approach, or just ask about a larger possible sample. 
- **Concerns**: Are the indices unique? What if we combine two tables that both use $0,1,2…n$ for the indices? 

In [1]:
# There should not be a need to install anything new 
import requests
import pandas as pd
from pathlib import Path 
import json

In [2]:
df = pd.DataFrame([["apple","morning"], 
                   ["pear","afternoon"], 
                   ["grapes","evening"]], 
                  index=["s1","s2","s3"], 
                  columns=["snack","time"])

display(df)

,snack,time
s1,apple,morning
s2,pear,afternoon
s3,grapes,evening


In [3]:
# adding a column: 

df["eval"] = ["great", "okay", "terrible"]
display(df)
tastes = {
    "s1": "sweet",
    "s2": "sour",
    "s3": "sweet"
}

df["tastes"] = tastes

display(df)

,snack,time,eval
s1,apple,morning,great
s2,pear,afternoon,okay
s3,grapes,evening,terrible


,snack,time,eval,tastes
s1,apple,morning,great,sweet
s2,pear,afternoon,okay,sour
s3,grapes,evening,terrible,sweet


In [5]:
# Concatenating DataFrames: 

df_2 = pd.DataFrame([["juice",], 
                   ["pear"], 
                   ["grapes"]], 
                  index=["s1","s2","s3"], 
                  columns=["drink"])

# Show a simple one column DataFrame
display(df_2)

# Show the catenation along the default axis
display(pd.concat([df, df_2], axis=0)) # axis=0 is the default

# Show the catenation along the other axis
display(pd.concat([df, df_2], axis=1))

,drink
s1,juice
s2,pear
s3,grapes


,snack,time,eval,tastes,drink
s1,apple,morning,great,sweet,NaN
s2,pear,afternoon,okay,sour,NaN
s3,grapes,evening,terrible,sweet,NaN
s1,NaN,NaN,NaN,NaN,juice
s2,NaN,NaN,NaN,NaN,pear
s3,NaN,NaN,NaN,NaN,grapes


,snack,time,eval,tastes,drink
s1,apple,morning,great,sweet,juice
s2,pear,afternoon,okay,sour,pear
s3,grapes,evening,terrible,sweet,grapes


# Increasing data: A one-to-many merge

Adding data is more interesting in cases where we either want to increase the amount of data or decrease it. Why might we want to do that? We increase data in the case where we have one entry in one table that pertains to multiple entries in the other table. For example, in the Stack Exchange data, we would have one row for each user in one table, yet, those users would post multiple comments or posts. 

This is normally structured in terms of hierarchies. For the purposes of this discussion, we can think of subordinate units and superordinate units. A subordinate unit is nested within a superordinate: cities are nested within countries so a superordinate row would be a country. That country row might contain a count of the number of cities. A city row might contain a column for the size of the national population.

To increase data: 
- **Operation**: A one to many merge (`pd.merge()`). 
- **Outcome**: Create new data for each row of the “many” units. 
- **Research purpose**: Account for contextual factors that would influence each subordinate unit (like being written by the same person).
- **Example**: Take the location field from each user row and copy it into the many rows where a user has made a comment. 
- **Follow-up**: Then we can filter by location across users, account for location in multilevel models. 
- **Concerns**: What about missing data? What if the data in one table has more rows than another column (i.e. users who don’t make posts but have registered or posts without a user as they were deleted in the data)

There are many details to consider in a merge, but the most important are:
- which data is left or right? 
- where are the keys in the left DataFrame?
- where are the keys in the right DataFrame? 

I will introduce aggregation before showing this code, since we will accomplish both within the same example. 

# Decreasing data: Grouping and aggregating 

- **Operation**: An aggregation (typically "Split-Apply-Combine")
- **Outcome**: Summarise the data from the many subordinate units and add it to a new column in the superordinate row. 
- **Research purpose**: Provide a comparative and often normalised perspective on measurements within superordinate units. 
- **Example**: A count of the number of times a person made a comment; normalised example: provide an average of the number of times a person made a comment that was upvoted. 
- **Follow-up**: Then we can use behavioural data about a person in order to make comparative claims between people rather than between observed units of behaviour. 
- **Concerns**: Aggregations like count, sum, and average might be done on groups of different sizes. When you aggregate are you comparing like for like?  

Again there are many details to consider in an aggregation, but the most important are:
- What method(s) are being used to summarise the data in your aggregation? 
- How are you handling missing data? 

# Practical example: Merging Social Network data
 
In the following data example, we have toy social network data from an example study. This data was created by Network Canvas. We will not be using it for collecting data, but it makes for a reasonable site for looking at this specific issue. 

In this case, I have created synthetic samples for 5 people. Each of these people nominated 3 to 5 of their closest friends. Then they responded how often they speak to these friends as well as how close they feel to these people and whether they live in the town where they grew up. The data will have some other columns in there. In networks studies there is typically also a table for links between people. We will not be covering that in the lecture, but will revisit this structure near the end of the course when we look at social networks. If you would like to see an example of this study yourself, you can go to this URL: This is a Network Canvas Fresco study that I have created. Fresco is a web service that one can deploy to conduct Network Canvas surveys.  

The first thing we will do is load both tables into Python and have a look at them in the Data Wrangler. These came from Network Canvas as `CSV` files, or comma separated values. The data also comes in a different format called GraphML. I have placed this in there for your own interest, but we will work with the CSV files today. 

Since they are comma separated values, this means that there’s no clear articulation of data type. They are all string values in the first instance, though we can clean the data by setting object types. I will do a minimal job of this, only sufficient to show the key operations. 

If you unzip this data folder in your data directory then it should be its own folder with six files in there: 
- `FSDSw1d3_EgoFile.csv`: This is the one table that contains measurements for the respondents. 
- `FSDSw1d3_AlterFile_1.csv` through `AlterFile_5`: These are the individual tables for each respondent’s alter nominations. 
- `FSDSw1d3_networkGraph.graphml`: This is the same data except it is in an XML format. 

Now in a network study we would normally have additional important data related to the connections between the people in any given personal network. In this case, I did not collect (read: simulate) the edge data for simplicity and brevity. 

Now imagine we have two different goals: 
1. To compare statistics between egos (i.e. respondents)
2. To compare statistics between alters, knowing they are nested within egos 

While this is a toy data set, these two operations would be common if you were comparing stats between users and stats between posts. 

This specific data structure will allow us to explore each of the three operations noted above: 
- Combine data: We will do this by combining the four Alter data tables. 
- Increase data: We will do this by adding data from the ego table to the alter table. This will be our one-to-many operation. 
- Decrease data: We will do this by aggregating data from the alter table and adding it to the ego table. Let’s have a look at each of these operations in turn. 




## Combining data: 

In Python this is typically done with a `pd.concat()` statement. One creates a collection of DataFrame objects and passes that collection to the concat statement. The method returns a new DataFrame with the old ones combined. It will not change the index, so we will need to reindex the data. 

Other things to note: 
1. I use `pathlib` for a more os-independent version of selecting the path, and then I select the files to import using `glob` (`glob` in this case stands for ‘global’, which is another word for "wildcard").  
2. Notice that I first create 5 individual DataFrames. Then I concatenate them. 
3. I display the resulting table only to show that the index is messed up. But then I ‘reindex’ the data. 
  - we have the option to either just create a generic index (0,1,2..n) but each row also has a special column called `networkCanvasUUID`. This is a unique string for each alter object. Since it is unique it would be useful as a _primary key_ and therefore I will use it for my index. Note that a primary key does not _need_ to be an index, but it is recommended. 




In [5]:
# Set the path to our folder. This time we use pathlib which is OS-agnostic
data_path = Path().cwd().parent / "data" / "network_data"

print(data_path)
print(data_path.exists())

if data_path.exists(): 
    network_files = list(data_path.glob("*AlterFile*"))
    
    print(f"There are {len(network_files)} files available.")


/Users/berniehogan/Documents/GitHub/fsds24code/data/network_data
True
There are 5 files available.


In [6]:
network_df_list = [pd.read_csv(file) for file in network_files]

alter_df = pd.concat(network_df_list) 

display(alter_df.head(8)) 

,nodeID,networkCanvasEgoUUID,networkCanvasUUID,name,ng_close,alter_live_hometown_1,alter_live_hometown_0,alter_live_hometown_-1,alter_live_now_1,alter_live_now_0,alter_live_now_-1,alter_contact_freq,null
0,1,50e482de-574d-4a4c-b9f4-3576a02603bf,35a8f929-74d2-400f-84d6-1a93a4a4936f,Leonardo,True,True,False,False,True,False,False,2,NaN
1,2,50e482de-574d-4a4c-b9f4-3576a02603bf,d15bd98a-b876-4ea3-b1c8-8de5f4f14096,Donatello,True,True,False,False,True,False,False,3,NaN
2,3,50e482de-574d-4a4c-b9f4-3576a02603bf,0d542a0b-58e2-4be3-94f5-1e57fd48a1d0,Raphael,True,True,False,False,True,False,False,2,NaN
3,4,50e482de-574d-4a4c-b9f4-3576a02603bf,331d81a6-6007-403c-9123-d258f09418fc,Michaelangelo,True,True,False,False,True,False,False,1,NaN
4,5,50e482de-574d-4a4c-b9f4-3576a02603bf,75371d13-1944-4928-88d2-9fd134cbc21d,April,True,False,True,False,True,False,False,1,NaN
0,1,42c648b3-36df-43da-9a45-bc27528d243f,369a9b50-510a-421b-b6df-99ad83a05f76,Daffy,True,False,True,False,False,True,False,2,NaN
1,2,42c648b3-36df-43da-9a45-bc27528d243f,96d1addb-44cb-4bc7-93ae-1f937a803cd3,bugs,True,True,False,False,False,True,False,3,NaN
2,3,42c648b3-36df-43da-9a45-bc27528d243f,e972ae78-fb8c-479e-8f97-4d4ef5bc2d01,Elmer,True,False,True,False,True,False,False,2,NaN


In [7]:
alter_df.set_index('networkCanvasUUID', inplace=True)

display(alter_df.head(8))

,nodeID,networkCanvasEgoUUID,name,ng_close,alter_live_hometown_1,alter_live_hometown_0,alter_live_hometown_-1,alter_live_now_1,alter_live_now_0,alter_live_now_-1,alter_contact_freq,null
networkCanvasUUID,,,,,,,,,,,,
35a8f929-74d2-400f-84d6-1a93a4a4936f,1,50e482de-574d-4a4c-b9f4-3576a02603bf,Leonardo,True,True,False,False,True,False,False,2,NaN
d15bd98a-b876-4ea3-b1c8-8de5f4f14096,2,50e482de-574d-4a4c-b9f4-3576a02603bf,Donatello,True,True,False,False,True,False,False,3,NaN
0d542a0b-58e2-4be3-94f5-1e57fd48a1d0,3,50e482de-574d-4a4c-b9f4-3576a02603bf,Raphael,True,True,False,False,True,False,False,2,NaN
331d81a6-6007-403c-9123-d258f09418fc,4,50e482de-574d-4a4c-b9f4-3576a02603bf,Michaelangelo,True,True,False,False,True,False,False,1,NaN
75371d13-1944-4928-88d2-9fd134cbc21d,5,50e482de-574d-4a4c-b9f4-3576a02603bf,April,True,False,True,False,True,False,False,1,NaN
369a9b50-510a-421b-b6df-99ad83a05f76,1,42c648b3-36df-43da-9a45-bc27528d243f,Daffy,True,False,True,False,False,True,False,2,NaN
96d1addb-44cb-4bc7-93ae-1f937a803cd3,2,42c648b3-36df-43da-9a45-bc27528d243f,bugs,True,True,False,False,False,True,False,3,NaN
e972ae78-fb8c-479e-8f97-4d4ef5bc2d01,3,42c648b3-36df-43da-9a45-bc27528d243f,Elmer,True,False,True,False,True,False,False,2,NaN


## Increasing data: 

Now with this data in a single table, we can add data to each row from the Ego table. For this we need a few operations: 

1. Ensure the ego data itself is in a DataFrame.
2. Determine what key will be used. In this case, I will again turn the unique ID into an index. In this case, it is called `networkCanvasEgoUUID`. 
3. Merge the data from the ego table into the alter table. For this we need the one key in the ego table that matches potentially many keys in the alter table. In this case, the Alter table has a column called EgoUUID. Therefore, we can use this to match the rows and merge in the data. 
	3.1. We do not need to merge in all columns from the EgoTable, only the columns that we consider relevant. Data is normalised since unnormalised data means lots of duplication. We want to acknowledge that and similarly minimise duplication. 
	3.2. In this case, I will merge in a single column: "Do you still live in your hometown?" with values `True` and `False`.

When we are done we should be able to view the new merged table which contains this data. If we look at the rows pertaining to a single respondent, all of the alters should have the same data in the new columns. 

Other things to note: 
- When we merge tables, the default kind of merge is ‘inner’, which means that if there are rows that do not match in either table they are excluded. There are many other kinds of ‘joins’, such as outer, left, inner, and cross-product. You are invited to explore these and they are written about in Chapter 5 of FSSTDS.


In [9]:
# Check for ego files in the folder
ego_files = list(data_path.glob("*EgoFile.csv"))

if len(ego_files) > 0: # Note: Fragile if you have more than one ego-file
    ego_file = ego_files[0]
    ego_df = pd.read_csv(ego_file)
else:
    print("The file was not found. Please ensure the ego file is in the correct folder.")    

display(ego_df.head())

ego_df.set_index("networkCanvasEgoUUID", inplace=True)

display(ego_df.head())


,networkCanvasEgoUUID,networkCanvasCaseID,networkCanvasSessionID,networkCanvasProtocolName,sessionStart,sessionFinish,sessionExported,ego_live_home
0,679e7d1d-6333-4d9d-aaed-4d4d0b04da08,case_001,95c87f16-78db-46ab-b978-222ba8dc9fe7,FSDS_ToyNetworkProtocol,2024-10-17T12:43:15.751Z,2024-10-17T12:44:02.899Z,2024-10-17T12:51:46.150Z,False
1,1c24befa-615e-4958-a61e-b35fa177eca3,case_002,cdfacd6e-a45a-44f3-9d6f-1a42dbb3754f,FSDS_ToyNetworkProtocol,2024-10-17T12:44:07.866Z,2024-10-17T12:45:24.011Z,2024-10-17T12:51:46.153Z,True
2,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,case_003,2652fd45-4f63-4e03-957b-e3dd018c5ecb,FSDS_ToyNetworkProtocol,2024-10-17T12:45:28.876Z,2024-10-17T12:47:27.130Z,2024-10-17T12:51:46.155Z,False
3,50e482de-574d-4a4c-b9f4-3576a02603bf,case_004,815d597a-f8da-467e-a190-1e56cf5a541f,FSDS_ToyNetworkProtocol,2024-10-17T12:47:32.258Z,2024-10-17T12:48:33.045Z,2024-10-17T12:51:46.160Z,True
4,42c648b3-36df-43da-9a45-bc27528d243f,case_005,cc5f8f3e-cf0e-44e7-bbb3-7c435dac64c4,FSDS_ToyNetworkProtocol,2024-10-17T12:48:37.362Z,2024-10-17T12:50:37.648Z,2024-10-17T12:51:46.170Z,False


,networkCanvasCaseID,networkCanvasSessionID,networkCanvasProtocolName,sessionStart,sessionFinish,sessionExported,ego_live_home
networkCanvasEgoUUID,,,,,,,
679e7d1d-6333-4d9d-aaed-4d4d0b04da08,case_001,95c87f16-78db-46ab-b978-222ba8dc9fe7,FSDS_ToyNetworkProtocol,2024-10-17T12:43:15.751Z,2024-10-17T12:44:02.899Z,2024-10-17T12:51:46.150Z,False
1c24befa-615e-4958-a61e-b35fa177eca3,case_002,cdfacd6e-a45a-44f3-9d6f-1a42dbb3754f,FSDS_ToyNetworkProtocol,2024-10-17T12:44:07.866Z,2024-10-17T12:45:24.011Z,2024-10-17T12:51:46.153Z,True
2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,case_003,2652fd45-4f63-4e03-957b-e3dd018c5ecb,FSDS_ToyNetworkProtocol,2024-10-17T12:45:28.876Z,2024-10-17T12:47:27.130Z,2024-10-17T12:51:46.155Z,False
50e482de-574d-4a4c-b9f4-3576a02603bf,case_004,815d597a-f8da-467e-a190-1e56cf5a541f,FSDS_ToyNetworkProtocol,2024-10-17T12:47:32.258Z,2024-10-17T12:48:33.045Z,2024-10-17T12:51:46.160Z,True
42c648b3-36df-43da-9a45-bc27528d243f,case_005,cc5f8f3e-cf0e-44e7-bbb3-7c435dac64c4,FSDS_ToyNetworkProtocol,2024-10-17T12:48:37.362Z,2024-10-17T12:50:37.648Z,2024-10-17T12:51:46.170Z,False


Now we have two tables, ego_df and alter_df. We have one column `ego_live_home` that we want to merge into our `alter_df` table. Recall the merge statement needs: 
- what data do we want to merge in (`ego_live_home`) 
- which table is left or right,
- and what are the keys. 

In this case, we want to link up the data based on the `networkCanvasEgoUUID` which is available in both tables. Review the merge statement below: 

In [10]:
alter_df_merged = alter_df.merge(ego_df[['ego_live_home']],
                                 left_on="networkCanvasEgoUUID", 
                                 right_index=True)
# Note that you can merge into the original alter_df but this would spell trouble if you do it twice. 

display(alter_df_merged.head())

,nodeID,networkCanvasEgoUUID,name,ng_close,alter_live_hometown_1,alter_live_hometown_0,alter_live_hometown_-1,alter_live_now_1,alter_live_now_0,alter_live_now_-1,alter_contact_freq,null,ego_live_home
networkCanvasUUID,,,,,,,,,,,,,
35a8f929-74d2-400f-84d6-1a93a4a4936f,1,50e482de-574d-4a4c-b9f4-3576a02603bf,Leonardo,True,True,False,False,True,False,False,2,NaN,True
d15bd98a-b876-4ea3-b1c8-8de5f4f14096,2,50e482de-574d-4a4c-b9f4-3576a02603bf,Donatello,True,True,False,False,True,False,False,3,NaN,True
0d542a0b-58e2-4be3-94f5-1e57fd48a1d0,3,50e482de-574d-4a4c-b9f4-3576a02603bf,Raphael,True,True,False,False,True,False,False,2,NaN,True
331d81a6-6007-403c-9123-d258f09418fc,4,50e482de-574d-4a4c-b9f4-3576a02603bf,Michaelangelo,True,True,False,False,True,False,False,1,NaN,True
75371d13-1944-4928-88d2-9fd134cbc21d,5,50e482de-574d-4a4c-b9f4-3576a02603bf,April,True,False,True,False,True,False,False,1,NaN,True


## Decreasing data: 

Strictly speaking we are not decreasing data that exists in the sense that we are always adding new measurements to some table. Aggregating data does not _necessarily_ destroy the old data. But we are decreasing the data _we are using_ by creating summaries of that data in some form and working with the summaries rather than the raw columns. 

This typically follows what Wickham has referred to as ‘Split-Apply-Combine’: Data is split into separate groups. Some aggregation is applied to each group individually, and then the results of that aggregation are then combined into a single data structure. 

We have already seen split-apply-combine implicitly since this is what a `value_counts()` does. It splits the data into groups, with one value in each group. It counts how many are in each group, and then combines these into a series that we can display or make use of subsequently. 

In this case, we will also do some counting, but I would like to spice this up with a little data transformation. We have measures for whether the person still lives in the town where they grew up. We also have measures for whether the alters live in the same place as ego. So we can now ask: how many of the contacts do ego have _on average_ that live in ego's home town? With only five cases we will not be able to draw any substantial conclusions, but we will see the sort of workflow that can be applied to much larger data.  

Notice that in the data we did not have a single variable for whether alter _currently_ lives in the hometown where they grew up. Instead we have 'did they grow up in the same town' and 'do they currently live in the same city as you'? But we can ask: 

- how many alters live in the same town as ego? 
- is there a difference between those who live in their hometown and those who live elsewhere? 

We can ask more complex questions as well (such as how many people moved to this town, do people contact those who they grew up with more than those who moved, etc..), but for now, we can appreciate the logic of aggregation just with these questions. 

First, let's aggregate the data by Ego. We will count the number of people who live in the same town as ego. For brevity, I will simply assert that there are two relevant variables: `alter_live_now` and `alter_live_hometown` that refer respectively to the questions: "do they live in the same city/town as you now" and "did they live in the same city/town as you when growing up"? These are given the possible choices [{"yes":1, "no":0, "I am unsure":-1}] which in this case became via the way the program exported data three variables each with True or False values. Thus, the column `alter_live_now_1` refers to the "yes" condition (i.e. they now live in the same town as the respondent). Thus, if we want to select those alters who live in the same town as ego, we would query for `df['alter_live_now_1' == True]`. Below see how we therefore create a count of this varible grouped by each different respondent and then added to the ego (i.e. respondent / superordinate level) table. 

We first _split_ the data into groups using a `groupby` statement. This creates a groupby object, which is not really useful on its own. But when we _apply_ some aggregation function to each group and then _combine_ the results to get a table with the aggregated values. Below see that in one line I group by `'networkCanvasEgoUUID'` (i.e. all the alters from a specific respondent) to create a new per-ego count of those who they talk to who live in their town. Then we do this a second time to group the users into 'lives in hometown' / 'lives elsewhere' count across egos.

The function that we apply can take different forms. One is by using a standard method, like `sum`, `min`, `max`, `count`. The other is by specifying a function per-column. There instead of `.sum()`, we could use `.agg({"col":"operation"})`, such as `.agg({"alter_live_now_1":"mean"})`. 

In [11]:
alter_score = alter_df.groupby("networkCanvasEgoUUID")["alter_live_now_1"].sum()

display(alter_score)

ego_df["alters_live_now"] = alter_score

display(ego_df)

networkCanvasEgoUUID
1c24befa-615e-4958-a61e-b35fa177eca3    3
2733bdae-69d9-47a4-bcbb-a981ba8c6b0c    5
42c648b3-36df-43da-9a45-bc27528d243f    2
50e482de-574d-4a4c-b9f4-3576a02603bf    5
679e7d1d-6333-4d9d-aaed-4d4d0b04da08    2
Name: alter_live_now_1, dtype: int64

,networkCanvasCaseID,networkCanvasSessionID,networkCanvasProtocolName,sessionStart,sessionFinish,sessionExported,ego_live_home,alters_live_now
networkCanvasEgoUUID,,,,,,,,
679e7d1d-6333-4d9d-aaed-4d4d0b04da08,case_001,95c87f16-78db-46ab-b978-222ba8dc9fe7,FSDS_ToyNetworkProtocol,2024-10-17T12:43:15.751Z,2024-10-17T12:44:02.899Z,2024-10-17T12:51:46.150Z,False,2
1c24befa-615e-4958-a61e-b35fa177eca3,case_002,cdfacd6e-a45a-44f3-9d6f-1a42dbb3754f,FSDS_ToyNetworkProtocol,2024-10-17T12:44:07.866Z,2024-10-17T12:45:24.011Z,2024-10-17T12:51:46.153Z,True,3
2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,case_003,2652fd45-4f63-4e03-957b-e3dd018c5ecb,FSDS_ToyNetworkProtocol,2024-10-17T12:45:28.876Z,2024-10-17T12:47:27.130Z,2024-10-17T12:51:46.155Z,False,5
50e482de-574d-4a4c-b9f4-3576a02603bf,case_004,815d597a-f8da-467e-a190-1e56cf5a541f,FSDS_ToyNetworkProtocol,2024-10-17T12:47:32.258Z,2024-10-17T12:48:33.045Z,2024-10-17T12:51:46.160Z,True,5
42c648b3-36df-43da-9a45-bc27528d243f,case_005,cc5f8f3e-cf0e-44e7-bbb3-7c435dac64c4,FSDS_ToyNetworkProtocol,2024-10-17T12:48:37.362Z,2024-10-17T12:50:37.648Z,2024-10-17T12:51:46.170Z,False,2


In [12]:
avg_ties_in_town = ego_df.groupby('ego_live_home').agg({"alters_live_now":"mean"})
avg_ties_in_town

,alters_live_now
ego_live_home,
False,3.0
True,4.0


If you are noticing carefully, we did not need to merge the ego data into the `alter_df` in order to examine this. These were two separate tasks. That said, they could be combined in all kinds of interesting ways. For example, in the `alter_df` table, we can now make a new column for those who moved to the respondent's hometown. That is, alters who did not grow up in the hometown (`'alter_live_hometown_0' == True`), but moved there (`alter_live_now_1 == True`). This only pertains if we know if Ego still lives in their hometown (`'ego_live_home' == True`). 

# Considering a data structure in the wild 

When we examine a data structure in the wild, we can see that it is often a mixed of nested objects. To make this more clear, we can observe reddit data. We will look more carefully at reddit later in the course. But for now we want to simply observe the structure of reddit data on the front page. 

When you look at the page as a rendered UX object, you can see all kinds of data structured as a series of posts. However, this data can also exist in a more minimal form that preserves the meaning but avoids all the aesthetic rendering. We can do that with any reddit page by simply replacing https://www.reddit.com with https://api.reddit.com/ 

When you go to that page, it might look like a mess or it might look like a series of nested objects, something like a file browser. This file is in JSON (JavaScript Object Notation). It is a common format that is not simply used with JavaScript. 

The file uses similar conventions to Python in the sense that {} represent key-value pairs (i.e. dictionaries) and [] represent sequences of data (i.e. lists). 

Any specific object within this structure could be a unit of analysis, or it could be somehow plausibly combined with our own preferred unit of analysis to help understand our object of inquiry. For example, some of the posts have images. We could _aggregate_ the number of images or we could _associate_ each image with the headline text for the post and have one row per image. 

In the interests of brevity, I will not be covering how to process such data today in depth. Instead, I merely want to show a very simple way in which we can reshape the reddit data, presently in JSON format, into a DataFrame. It will not work perfectly because the data is so nested. You will see this when we look at the resulting DataFrame. But it does help to highlight how we transform data from a nested format into a rectangular format. 

We start by collecting the data in JSON format. We will see more of this next week when we look at accessing data on the web. 

In [15]:
# Set a unique user-agent string
headers = {
    'User-Agent': 'MyRedditExampleScript/1.0 (by berniehogan)'
}

# Make a GET request to the Reddit API
url = 'https://api.reddit.com/r/popular'
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = json.loads(response.text)
    
    # Use json_normalize to create a DataFrame
    df = pd.json_normalize(data['data']['children'])
    
    # Display the first few rows of the DataFrame
    display(df.head())
    print(f"There are a total of {len(df)} records in the DataFrame.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


,kind,data.approved_at_utc,data.subreddit,data.selftext,data.author_fullname,data.saved,data.mod_reason_title,data.gilded,data.clicked,data.title,...,data.media_metadata.obnlp5ilebvd1.s.u,data.media_metadata.obnlp5ilebvd1.id,data.media_metadata.5td9a5glebvd1.status,data.media_metadata.5td9a5glebvd1.e,data.media_metadata.5td9a5glebvd1.m,data.media_metadata.5td9a5glebvd1.p,data.media_metadata.5td9a5glebvd1.s.y,data.media_metadata.5td9a5glebvd1.s.x,data.media_metadata.5td9a5glebvd1.s.u,data.media_metadata.5td9a5glebvd1.id
0,t3,None,WatchPeopleDieInside,,t2_wziv9qfyn,False,None,0,False,Racist asks Canadian to go Back to India becau...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t3,None,MadeMeSmile,,t2_17bmi8,False,None,0,False,Dave Bautista explaining the GOP Presidential ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,t3,None,HumansBeingBros,,t2_ppdx73y6,False,None,0,False,Small dog rescued from flood waters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,t3,None,politics,,t2_35fk2,False,None,0,False,Donald Trump Cancels Second Mainstream Intervi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,t3,None,politics,,t2_nueh0i5gq,False,None,0,False,Kamala Harris 'Dominated' Bret Baier in Fox Ne...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are a total of 25 records in the DataFrame.


# Other forms of data wrangling: A quick survey 

There are many other forms of wrangling to consider, but a few are worth mentioning here: 
1. Transposing. This is how we flip data so that what was in the rows is now in the columns and vice versa. In a matrix or a DataFrame it's very simple: `<object>.T`. Observe `alter_df.T`

In [16]:
display(alter_df.T)

networkCanvasUUID,35a8f929-74d2-400f-84d6-1a93a4a4936f,d15bd98a-b876-4ea3-b1c8-8de5f4f14096,0d542a0b-58e2-4be3-94f5-1e57fd48a1d0,331d81a6-6007-403c-9123-d258f09418fc,75371d13-1944-4928-88d2-9fd134cbc21d,369a9b50-510a-421b-b6df-99ad83a05f76,96d1addb-44cb-4bc7-93ae-1f937a803cd3,e972ae78-fb8c-479e-8f97-4d4ef5bc2d01,e8db54fc-ae5f-4988-960f-f55399bf240c,cec31140-3c79-4598-993b-944b05e5183a,...,e315374d-3aef-48e4-8e55-3c55ee8d6e3c,8eff9bed-95f7-4bf5-9dbd-e8fb5cd269db,ebc2226d-982b-443c-8655-5fb7db7dbdf3,ae86fbdb-9424-4d74-9f17-abb50f106ddd,c76b4b4d-ada6-4529-b340-a7ad99ce0c70,f4fc9377-0a5f-4fd3-8c70-562ed99791a1,fb229e12-4660-43bd-92a1-d5dbaaabdf60,8c0d3bfb-95ab-480c-b5b9-467419c434f4,35dfe21a-6f04-4cca-bedc-816fb5a5c8ab,2801ff25-9d53-434e-9a89-f813e6afda01
nodeID,1,2,3,4,5,1,2,3,4,5,...,4,1,2,3,4,1,2,3,4,5
networkCanvasEgoUUID,50e482de-574d-4a4c-b9f4-3576a02603bf,50e482de-574d-4a4c-b9f4-3576a02603bf,50e482de-574d-4a4c-b9f4-3576a02603bf,50e482de-574d-4a4c-b9f4-3576a02603bf,50e482de-574d-4a4c-b9f4-3576a02603bf,42c648b3-36df-43da-9a45-bc27528d243f,42c648b3-36df-43da-9a45-bc27528d243f,42c648b3-36df-43da-9a45-bc27528d243f,42c648b3-36df-43da-9a45-bc27528d243f,42c648b3-36df-43da-9a45-bc27528d243f,...,679e7d1d-6333-4d9d-aaed-4d4d0b04da08,1c24befa-615e-4958-a61e-b35fa177eca3,1c24befa-615e-4958-a61e-b35fa177eca3,1c24befa-615e-4958-a61e-b35fa177eca3,1c24befa-615e-4958-a61e-b35fa177eca3,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c,2733bdae-69d9-47a4-bcbb-a981ba8c6b0c
name,Leonardo,Donatello,Raphael,Michaelangelo,April,Daffy,bugs,Elmer,Foghorn,Porky,...,Homer,Bert,Ernie,Oscar,Kermit,Alice,Hermione,Frodo,Bilbo,Harry
ng_close,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
alter_live_hometown_1,True,True,True,True,False,False,True,False,False,True,...,True,False,False,True,False,False,False,True,True,False
alter_live_hometown_0,False,False,False,False,True,True,False,True,False,False,...,False,True,True,False,True,True,True,False,False,True
alter_live_hometown_-1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
alter_live_now_1,True,True,True,True,True,False,False,True,True,False,...,True,True,True,True,False,True,True,True,True,True
alter_live_now_0,False,False,False,False,False,True,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
alter_live_now_-1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


The second operation is how we take a list of data and make it into columns. This is more pertinent with the Stack Exchange data. In that data there is a column for tags. This is there twice, once as a string: "<movie><cinematic><etc...>" and once as a list: `['movie', 'cinematic', 'etc...']`. When it is a list that means we could in theory have rows per-tag rather than tags per post. This is an example of transforming _wide_ data into _long_ data, but from our point of view in terms of research, what we are doing is changing the unit of analysis from posts to tags. Using a toy example, we can see that below using the `explode()` function:

In [17]:
# Create the sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan'],
    'Age': [25, 30, 22, 28, 35],
    'Favourite_Flavours': [
        ['Vanilla', 'Chocolate'],
        ['Strawberry', 'Mint'],
        ['Chocolate', 'Cookie Dough', 'Vanilla'],
        ['Mint', 'Pistachio'],
        ['Vanilla', 'Strawberry', 'Chocolate']
    ]
}

df = pd.DataFrame(data)

print("Original DataFrame:")
print(df)
print("\n")

df_long = df.explode('Favourite_Flavours')

print("Exploded DataFrame:")
print(df_long)

Original DataFrame:
      Name  Age                  Favourite_Flavours
0    Alice   25                [Vanilla, Chocolate]
1      Bob   30                  [Strawberry, Mint]
2  Charlie   22  [Chocolate, Cookie Dough, Vanilla]
3    Diana   28                   [Mint, Pistachio]
4    Ethan   35    [Vanilla, Strawberry, Chocolate]


Exploded DataFrame:
      Name  Age Favourite_Flavours
0    Alice   25            Vanilla
0    Alice   25          Chocolate
1      Bob   30         Strawberry
1      Bob   30               Mint
2  Charlie   22          Chocolate
2  Charlie   22       Cookie Dough
2  Charlie   22            Vanilla
3    Diana   28               Mint
3    Diana   28          Pistachio
4    Ethan   35            Vanilla
4    Ethan   35         Strawberry
4    Ethan   35          Chocolate


# Examining the Stack Exchange data

Whereas in this example we had people who nominated their friends. But we can merge tables for all kinds of reasons. The lab for today is a pretty tricky one, to be honest, but it is based on an enduring concept of 'social roles' in online spaces. That is, people often adopt a role which is a form of behaviour in relation to a specific other set of people. Some people might only ever ask questions on a forum, others might only answer. 

We want to find out a little something about the people based on their roles. We might ask 'who is more invested' in their profile on the forum: the people who post or the people who reply. We might ask more complex questions as well. In the assignment, we will start with that simple question and then ask a more complex question related to roles. Both questions will require you to merge data. 

The original Stack Exchange data was called `movies_stack_df.feather`. The new one will be called `movies_stack_df_users.feather`. It can be loaded and examined in the same manner as the original data. I would recommend that instead of calling it `stack_df` that now you call it `posts_df` and `users_df` for clarity. 

You will see extenxive instructions to help out. 


# AI Disclosure

In this analysis, Claude Sonnet 3.5 was used to help assist with the `explode` command. That said, it took several iterations to simplify down to the code as seen, whereby I needed to refine the code by hand. Notably Claude remarked "Your solution is indeed the most elegant for this specific case". Note here that I was seeking elegance not efficiency or robustness, since I wanted clear code for readability.  

It also tidied up my requests statement to add a response code error exception, but it did not parse the json correctly, which had to be done by hand, both extracting from the requests response but also during the json_normalize().

Here was Claude's original take on the explode section which was far too complicated (and used American spelling):

In [ ]:
# Create the sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan'],
    'Age': [25, 30, 22, 28, 35],
    'Favorite_Flavors': [
        ['Vanilla', 'Chocolate'],
        ['Strawberry', 'Mint'],
        ['Chocolate', 'Cookie Dough', 'Vanilla'],
        ['Mint', 'Pistachio'],
        ['Vanilla', 'Strawberry', 'Chocolate']
    ]
}

df = pd.DataFrame(data)

print("Original DataFrame:")
print(df)
print("\n")

# Convert to wide format
df_wide = df.explode('Favorite_Flavors').pivot_table(
    index=['Name', 'Age'],
    columns='Favorite_Flavors',
    aggfunc=lambda x: 1,
    fill_value=0
).reset_index()

# Flatten column names
df_wide.columns.name = None
df_wide.columns = ['Name', 'Age'] + list(df_wide.columns[2:])

print("Wide Format DataFrame:")
print(df_wide)